### This notebook Re-formats the KEGG KO annotations "master file" from [GhostKoalaParser github page](https://github.com/edgraham/GhostKoalaParser/tree/master/samples) into 2-column tables that can be used with [Enveomics scripts](https://github.com/lmrodriguezr/enveomics/tree/master/Scripts)

##### To do Later: converts assembly gene IDs to KOs and KO functions and estimates their coverage in a count matrix for DESEQ2 analysis 
1. First map MG short reads to ORFs from the corresponding assembly using MagicBlast. Filter the output for best hit, 90% query cover and 50bp query length. Convert the outputs to 2-column count tables where col1 = geneID and col2= #reads mapping
2. Annotate the ORFs using KoFamScan

In [2]:
import os as os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob as glob
from scipy import stats

### Format the master KO file for easy parsing with the read count tables

In [4]:
## import the master KO htext file
#os.chdir("../")
ko_master = pd.read_table("01.KEGG_MetaTables/KO_Orthology_ko00001.txt", sep="\t",names=['Group', 'Subgroup1', 'Subgroup2', 'KO'])
ko_master.head()

,Group,Subgroup1,Subgroup2,KO
0,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K00844 HK; hexokinase [EC:2.7.1.1]
1,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K12407 GCK; glucokinase [EC:2.7.1.2]
2,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K00845 glk; glucokinase [EC:2.7.1.2]
3,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K00886 ppgK; polyphosphate glucokinase [EC:2....
4,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K08074 ADPGK; ADP-dependent glucokinase [EC:2...


In [5]:
## Split the 'KO' column into KO and FUCNTION
ko_master[['KO', 'Function']]= ko_master['KO'].str.split(" ",n=1, expand=True)
ko_master.head()

,Group,Subgroup1,Subgroup2,KO,Function
0,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K00844,HK; hexokinase [EC:2.7.1.1]
1,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K12407,GCK; glucokinase [EC:2.7.1.2]
2,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K00845,glk; glucokinase [EC:2.7.1.2]
3,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K00886,ppgK; polyphosphate glucokinase [EC:2.7.1.63]
4,Metabolism,Overview,01200 Carbon metabolism [PATH:ko01200],K08074,ADPGK; ADP-dependent glucokinase [EC:2.7.1.147]


In [6]:
## Split out and REMOVE the "PATH" part of Subgroup2 label
ko_master[['Subgroup2', 'PATH']] = ko_master['Subgroup2'].str.split("[", n=1, expand=True)
ko_master = ko_master.drop('PATH', axis=1)
ko_master.head()

,Group,Subgroup1,Subgroup2,KO,Function
0,Metabolism,Overview,01200 Carbon metabolism,K00844,HK; hexokinase [EC:2.7.1.1]
1,Metabolism,Overview,01200 Carbon metabolism,K12407,GCK; glucokinase [EC:2.7.1.2]
2,Metabolism,Overview,01200 Carbon metabolism,K00845,glk; glucokinase [EC:2.7.1.2]
3,Metabolism,Overview,01200 Carbon metabolism,K00886,ppgK; polyphosphate glucokinase [EC:2.7.1.63]
4,Metabolism,Overview,01200 Carbon metabolism,K08074,ADPGK; ADP-dependent glucokinase [EC:2.7.1.147]


In [7]:
#Split Subgroup2 and its numeric code
#convert to string, split on the FIRST space only (n=1), create a new column over splits (expand=True) 
ko_master[['S2 code','Subgroup2']]=ko_master['Subgroup2'].str.split(" ", n=1, expand=True)
ko_master.head()

,Group,Subgroup1,Subgroup2,KO,Function,S2 code
0,Metabolism,Overview,Carbon metabolism,K00844,HK; hexokinase [EC:2.7.1.1],01200
1,Metabolism,Overview,Carbon metabolism,K12407,GCK; glucokinase [EC:2.7.1.2],01200
2,Metabolism,Overview,Carbon metabolism,K00845,glk; glucokinase [EC:2.7.1.2],01200
3,Metabolism,Overview,Carbon metabolism,K00886,ppgK; polyphosphate glucokinase [EC:2.7.1.63],01200
4,Metabolism,Overview,Carbon metabolism,K08074,ADPGK; ADP-dependent glucokinase [EC:2.7.1.147],01200


In [20]:
#Unique items for each Group and SubGroup
print(ko_master.Group.unique())
print(ko_master.Subgroup1.unique()) ## 43 different Subgroup1 categories
len(ko_master.Subgroup2.unique()) ## 430 different Subgroup2 categories
SG2 = ko_master.Subgroup2.unique()
SG2[:10]

['Metabolism' 'Genetic Information Processing'
 'Environmental Information Processing' 'Cellular Processes'
 'Organismal Systems' 'Human Diseases']
['Overview' 'Carbohydrate metabolism' 'Energy metabolism'
 'Lipid metabolism' 'Nucleotide metabolism' 'Amino acid metabolism'
 'Metabolism of other amino acids' 'Glycan biosynthesis and metabolism'
 'Metabolism of cofactors and vitamins'
 'Metabolism of terpenoids and polyketides'
 'Biosynthesis of other secondary metabolites'
 'Xenobiotics biodegradation and metabolism' 'Enzyme families'
 'Transcription' 'Translation' 'Folding, sorting and degradation'
 'Replication and repair' 'Membrane transport' 'Signal transduction'
 'Signaling molecules and interaction' 'Transport and catabolism'
 'Cell growth and death' 'Cellular community - eukaryotes'
 'Cellular community - prokaryotes' 'Cell motility' 'Immune system'
 'Endocrine system' 'Circulatory system' 'Digestive system'
 'Excretory system' 'Nervous system' 'Sensory system' 'Development' 'Agi

array(['Carbon metabolism ', '2-Oxocarboxylic acid metabolism ',
       'Fatty acid metabolism ', 'Biosynthesis of amino acids ',
       'Degradation of aromatic compounds ',
       'Glycolysis / Gluconeogenesis ', 'Citrate cycle (TCA cycle) ',
       'Pentose phosphate pathway ',
       'Pentose and glucuronate interconversions ',
       'Fructose and mannose metabolism '], dtype=object)

In [243]:
## export formatted table to csv file
ko_master.to_csv("KO_Orthology_reformated.txt", sep="\t")

In [248]:
## Make metadata tables for Miguel's table summary and merge scripts
## First column is KO, second column is Subgroup1 
KO_Group = ko_master[["KO","Group"]]
KO_SubGroup1 = ko_master[["KO","Subgroup1"]]
KO_SubGroup2 = ko_master[["KO","Subgroup2"]]
KO_Function = ko_master[["KO","Function"]]

KO_Group.to_csv("KO_Group.txt", sep="\t", index=False)
KO_SubGroup1.to_csv("KO_SubGroup1.txt", sep="\t", index=False)
KO_SubGroup2.to_csv("KO_SubGroup2.txt", sep="\t", index=False)
KO_Function.to_csv("KO_Function.txt", sep="\t", index=False)

### This is to-do later! Format the ReadMap and KO annoations in python.

In [44]:
## Import the  blast count tables
read_counts = pd.read_csv("Assembly_ReadMap_tables/hum2_readmap.fltrdBstHts.blst.table",sep="\t", names=['ORF','coverage'])
print(read_counts.head(2))

## Import the gene to KEGG annotation tables as dictionaries remove the rows that do NOT have KO
KO_annotation = pd.read_csv("Assembly_KO_annotation_tables/hum2.kofams.txt", sep="\t", names=['ORF','KO']).dropna()
print(KO_annotation.head(2))
# convert df to dictionary
KO_dict = KO_annotation.set_index('ORF')['KO'].to_dict()

## Convert the geneIDs to KO numbers in each read_counts table
## remove the ORFs that DO not have a KO assigned

## Merge the tables together to create a KO count matrix with Samples as the column and KOs as rows

                 ORF  coverage
0  hum2_gene_10420_1        36
1   hum2_gene_115_30         1
             ORF      KO
0  hum2_gene_1_1  K03327
3  hum2_gene_1_4  K03183


In [6]:
## Create a dictionary with the ORF-KEGG annotation files (*.kofams.txt):
# only need to run the os.chdir command once
os.chdir("Assembly_KO_annotation_tables/")
print(os.getcwd())
KO_Dict={} 
files = [f for f in os.listdir() if not f.startswith('.')]
for file in files:
    #print(file)
    with open(file) as KO_file: 
        for line in KO_file:
            line = line.rstrip("\n")
            line = line.split("\t")
            if len(line) == 2:
                ORF = line[0]
                KO = line[1]
            KO_Dict[ORF] = KO
print(KO_Dict)           

/Users/brittanysuttner/Desktop/KOs_MAGs_ttest_clustermap/Assembly_KO_annotation_tables
{'hum2_gene_1_1': 'K03327', 'hum2_gene_1_4': 'K03183', 'hum2_gene_1_5': 'K02004', 'hum2_gene_1_6': 'K16066', 'hum2_gene_1_7': 'K06973', 'hum2_gene_1_9': 'K19294', 'hum2_gene_1_11': 'K08159', 'hum2_gene_1_13': 'K04567', 'hum2_gene_1_14': 'K01803', 'hum2_gene_1_15': 'K06001', 'hum2_gene_1_16': 'K02879', 'hum2_gene_1_17': 'K03040', 'hum2_gene_1_18': 'K02986', 'hum2_gene_1_19': 'K02948', 'hum2_gene_1_20': 'K02952', 'hum2_gene_1_21': 'K02919', 'hum2_gene_1_22': 'K02518', 'hum2_gene_1_23': 'K01265', 'hum2_gene_1_24': 'K03076', 'hum2_gene_1_25': 'K02876', 'hum2_gene_1_26': 'K02907', 'hum2_gene_1_27': 'K02988', 'hum2_gene_1_28': 'K02881', 'hum2_gene_1_29': 'K02933', 'hum2_gene_1_30': 'K02994', 'hum2_gene_1_31': 'K02954', 'hum2_gene_1_32': 'K02931', 'hum2_gene_1_33': 'K02895', 'hum2_gene_1_34': 'K02874', 'hum2_gene_1_35': 'K02961', 'hum2_gene_1_36': 'K02904', 'hum2_gene_1_37': 'K02878', 'hum2_gene_1_38': 'K02